In [1]:
import sys
import numpy as np
sys.path.append(r'C:\Users\admin\Documents\NiDyN_EEGNET\arl-eegmodels')
from EEGModels import EEGNet, ShallowConvNet, DeepConvNet
from scipy.io import loadmat
from tensorflow.keras.optimizers import Adam
import glob
file_list = glob.glob("Processed_Data\*norm.mat")

In [2]:
subj = 1

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1887 samples, validate on 112 samples
Epoch 1/200
1887/1887 [==============================] - 4s 2ms/step - loss: 1.0559 - acc: 0.4510 - val_loss: 0.7068 - val_acc: 0.3661
Epoch 2/200
1887/1887 [==============================] - 3s 1ms/step - loss: 1.0353 - acc: 0.4812 - val_loss: 0.7447 - val_acc: 0.4018
Epoch 3/200
1887/1887 [==============================] - 3s 1ms/step - loss: 1.0058 - acc: 0.5639 - val_loss: 0.7459 - val_acc: 0.4464
Epoch 4/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.9766 - acc: 0.5967 - val_loss: 0.7461 - val_acc: 0.5000
Epoch 5/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.9520 - acc: 0.6275 - val_loss: 0.7693 - val_acc: 0.5000
Epoch 6/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.9550 - acc: 0.6306 - val_loss: 0.7550 - val_acc: 0.5357
Epoch 7/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.9348 - acc: 0.6460 - val_loss: 0.7657 - val_acc: 0.5268
Epoch 8/

Epoch 61/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.7298 - acc: 0.7541 - val_loss: 0.9142 - val_acc: 0.5536
Epoch 62/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.7327 - acc: 0.7472 - val_loss: 0.8612 - val_acc: 0.5982
Epoch 63/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.7149 - acc: 0.7573 - val_loss: 0.9262 - val_acc: 0.5446
Epoch 64/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.7213 - acc: 0.7658 - val_loss: 0.8420 - val_acc: 0.6161
Epoch 65/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.7248 - acc: 0.7520 - val_loss: 0.9183 - val_acc: 0.5625
Epoch 66/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.7355 - acc: 0.7562 - val_loss: 0.9724 - val_acc: 0.5357
Epoch 67/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.7298 - acc: 0.7467 - val_loss: 1.0150 - val_acc: 0.4911
Epoch 68/200
1887/1887 [========================

1887/1887 [==============================] - 3s 2ms/step - loss: 0.6966 - acc: 0.7742 - val_loss: 1.0160 - val_acc: 0.5179
Epoch 122/200
1887/1887 [==============================] - 3s 2ms/step - loss: 0.6964 - acc: 0.7636 - val_loss: 0.9976 - val_acc: 0.5446
Epoch 123/200
1887/1887 [==============================] - 3s 2ms/step - loss: 0.6847 - acc: 0.7748 - val_loss: 0.9809 - val_acc: 0.5536
Epoch 124/200
1887/1887 [==============================] - 4s 2ms/step - loss: 0.6787 - acc: 0.7716 - val_loss: 0.9341 - val_acc: 0.5536
Epoch 125/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6752 - acc: 0.7795 - val_loss: 1.0121 - val_acc: 0.5446
Epoch 126/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6801 - acc: 0.7647 - val_loss: 0.9181 - val_acc: 0.6071
Epoch 127/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6938 - acc: 0.7695 - val_loss: 0.9261 - val_acc: 0.5893
Epoch 128/200
1887/1887 [==============================

Epoch 181/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6555 - acc: 0.7817 - val_loss: 0.9528 - val_acc: 0.6161
Epoch 182/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6488 - acc: 0.7880 - val_loss: 0.9217 - val_acc: 0.6161
Epoch 183/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6385 - acc: 0.7917 - val_loss: 0.9164 - val_acc: 0.6250
Epoch 184/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6600 - acc: 0.7700 - val_loss: 0.8968 - val_acc: 0.6161
Epoch 185/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6358 - acc: 0.7833 - val_loss: 0.9099 - val_acc: 0.6161
Epoch 186/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6562 - acc: 0.7827 - val_loss: 0.9674 - val_acc: 0.6250
Epoch 187/200
1887/1887 [==============================] - 3s 1ms/step - loss: 0.6190 - acc: 0.8039 - val_loss: 0.9371 - val_acc: 0.5982
Epoch 188/200
1887/1887 [================

111/111 [==============================] - 0s 2ms/step - loss: 0.3363 - acc: 0.9189 - val_loss: 0.7437 - val_acc: 0.6250
Epoch 42/100
111/111 [==============================] - 0s 2ms/step - loss: 0.3353 - acc: 0.9369 - val_loss: 0.7352 - val_acc: 0.6429
Epoch 43/100
111/111 [==============================] - 0s 2ms/step - loss: 0.3691 - acc: 0.9099 - val_loss: 0.7295 - val_acc: 0.6518
Epoch 44/100
111/111 [==============================] - 0s 2ms/step - loss: 0.3808 - acc: 0.9189 - val_loss: 0.7411 - val_acc: 0.6339
Epoch 45/100
111/111 [==============================] - 0s 2ms/step - loss: 0.3304 - acc: 0.9550 - val_loss: 0.7409 - val_acc: 0.6339
Epoch 46/100
111/111 [==============================] - 0s 2ms/step - loss: 0.3415 - acc: 0.9189 - val_loss: 0.7368 - val_acc: 0.6339
Epoch 47/100
111/111 [==============================] - 0s 2ms/step - loss: 0.3767 - acc: 0.8919 - val_loss: 0.7356 - val_acc: 0.6339
Epoch 48/100
111/111 [==============================] - 0s 2ms/step - loss:

In [3]:
subj = 2

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1964 samples, validate on 74 samples
Epoch 1/200
1964/1964 [==============================] - 4s 2ms/step - loss: 1.0460 - acc: 0.5046 - val_loss: 0.6946 - val_acc: 0.5270
Epoch 2/200
1964/1964 [==============================] - 3s 1ms/step - loss: 1.0112 - acc: 0.5530 - val_loss: 0.6456 - val_acc: 0.6757
Epoch 3/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.9906 - acc: 0.5810 - val_loss: 0.6234 - val_acc: 0.6757
Epoch 4/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.9709 - acc: 0.5973 - val_loss: 0.5924 - val_acc: 0.7973
Epoch 5/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.9503 - acc: 0.6191 - val_loss: 0.5831 - val_acc: 0.7027
Epoch 6/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.9438 - acc: 0.6477 - val_loss: 0.5973 - val_acc: 0.7027
Epoch 7/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.9241 - acc: 0.6568 - val_loss: 0.6278 - val_acc: 0.6757
Epoch 8/2

Epoch 61/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.7391 - acc: 0.7617 - val_loss: 0.6234 - val_acc: 0.6486
Epoch 62/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.7275 - acc: 0.7643 - val_loss: 0.5994 - val_acc: 0.6486
Epoch 63/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.7249 - acc: 0.7729 - val_loss: 0.6455 - val_acc: 0.6081
Epoch 64/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.7357 - acc: 0.7648 - val_loss: 0.6618 - val_acc: 0.5946
Epoch 65/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.7227 - acc: 0.7693 - val_loss: 0.6586 - val_acc: 0.5946
Epoch 66/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.7320 - acc: 0.7617 - val_loss: 0.5932 - val_acc: 0.6622
Epoch 67/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.7469 - acc: 0.7673 - val_loss: 0.6412 - val_acc: 0.6351
Epoch 68/200
1964/1964 [========================

1964/1964 [==============================] - 3s 1ms/step - loss: 0.6989 - acc: 0.7805 - val_loss: 0.7645 - val_acc: 0.5541
Epoch 122/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6932 - acc: 0.7770 - val_loss: 0.6738 - val_acc: 0.6486
Epoch 123/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6859 - acc: 0.7841 - val_loss: 0.7043 - val_acc: 0.5946
Epoch 124/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6992 - acc: 0.7897 - val_loss: 0.7260 - val_acc: 0.5811
Epoch 125/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6831 - acc: 0.7846 - val_loss: 0.7341 - val_acc: 0.5811
Epoch 126/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6803 - acc: 0.7882 - val_loss: 0.7402 - val_acc: 0.6081
Epoch 127/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6698 - acc: 0.7923 - val_loss: 0.6813 - val_acc: 0.5946
Epoch 128/200
1964/1964 [==============================

Epoch 181/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6373 - acc: 0.7923 - val_loss: 0.6795 - val_acc: 0.6622
Epoch 182/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6567 - acc: 0.7948 - val_loss: 0.7125 - val_acc: 0.6081
Epoch 183/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6964 - acc: 0.7826 - val_loss: 0.8099 - val_acc: 0.5946
Epoch 184/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6601 - acc: 0.7943 - val_loss: 0.7849 - val_acc: 0.6216
Epoch 185/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6675 - acc: 0.7805 - val_loss: 0.6320 - val_acc: 0.6892
Epoch 186/200
1964/1964 [==============================] - 3s 2ms/step - loss: 0.6524 - acc: 0.7943 - val_loss: 0.6691 - val_acc: 0.6216
Epoch 187/200
1964/1964 [==============================] - 3s 1ms/step - loss: 0.6681 - acc: 0.7963 - val_loss: 0.6819 - val_acc: 0.5946
Epoch 188/200
1964/1964 [================

72/72 [==============================] - 0s 2ms/step - loss: 0.4559 - acc: 0.9028 - val_loss: 0.5923 - val_acc: 0.7162
Epoch 43/100
72/72 [==============================] - 0s 2ms/step - loss: 0.4180 - acc: 0.9167 - val_loss: 0.5906 - val_acc: 0.7297
Epoch 44/100
72/72 [==============================] - 0s 2ms/step - loss: 0.4270 - acc: 0.9167 - val_loss: 0.5910 - val_acc: 0.7297
Epoch 45/100
72/72 [==============================] - 0s 2ms/step - loss: 0.4415 - acc: 0.9028 - val_loss: 0.5909 - val_acc: 0.7162
Epoch 46/100
72/72 [==============================] - 0s 2ms/step - loss: 0.5618 - acc: 0.8750 - val_loss: 0.5930 - val_acc: 0.7027
Epoch 47/100
72/72 [==============================] - 0s 2ms/step - loss: 0.4811 - acc: 0.9028 - val_loss: 0.5941 - val_acc: 0.7027
Epoch 48/100
72/72 [==============================] - 0s 2ms/step - loss: 0.4014 - acc: 0.9167 - val_loss: 0.5952 - val_acc: 0.7027
Epoch 49/100
72/72 [==============================] - ETA: 0s - loss: 0.3314 - acc: 0.921

In [4]:
subj = 3

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1864 samples, validate on 124 samples
Epoch 1/200
1864/1864 [==============================] - 4s 2ms/step - loss: 1.0519 - acc: 0.4619 - val_loss: 0.6928 - val_acc: 0.5000
Epoch 2/200
1864/1864 [==============================] - 3s 1ms/step - loss: 1.0259 - acc: 0.5225 - val_loss: 0.6972 - val_acc: 0.5161
Epoch 3/200
1864/1864 [==============================] - 3s 1ms/step - loss: 1.0011 - acc: 0.5477 - val_loss: 0.7289 - val_acc: 0.4677
Epoch 4/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.9763 - acc: 0.6003 - val_loss: 0.7086 - val_acc: 0.5484
Epoch 5/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.9534 - acc: 0.6266 - val_loss: 0.7116 - val_acc: 0.5403
Epoch 6/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.9432 - acc: 0.6325 - val_loss: 0.7054 - val_acc: 0.6129
Epoch 7/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.9291 - acc: 0.6411 - val_loss: 0.7135 - val_acc: 0.5323
Epoch 8/

Epoch 61/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.7243 - acc: 0.7709 - val_loss: 0.6800 - val_acc: 0.6290
Epoch 62/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.7208 - acc: 0.7881 - val_loss: 0.7016 - val_acc: 0.6048
Epoch 63/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.7521 - acc: 0.7591 - val_loss: 0.7294 - val_acc: 0.5887
Epoch 64/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.7228 - acc: 0.7698 - val_loss: 0.7569 - val_acc: 0.5726
Epoch 65/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.7215 - acc: 0.7715 - val_loss: 0.7199 - val_acc: 0.6048
Epoch 66/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.7230 - acc: 0.7741 - val_loss: 0.7136 - val_acc: 0.6210
Epoch 67/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.7154 - acc: 0.7575 - val_loss: 0.7399 - val_acc: 0.5968
Epoch 68/200
1864/1864 [========================

1864/1864 [==============================] - 3s 2ms/step - loss: 0.6718 - acc: 0.7731 - val_loss: 0.7777 - val_acc: 0.5887
Epoch 122/200
1864/1864 [==============================] - 3s 2ms/step - loss: 0.6798 - acc: 0.7892 - val_loss: 0.6950 - val_acc: 0.6048
Epoch 123/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6685 - acc: 0.7892 - val_loss: 0.7922 - val_acc: 0.5887
Epoch 124/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6622 - acc: 0.7940 - val_loss: 0.8155 - val_acc: 0.5726
Epoch 125/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6935 - acc: 0.7709 - val_loss: 0.8007 - val_acc: 0.5968
Epoch 126/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6373 - acc: 0.8020 - val_loss: 0.8426 - val_acc: 0.5806
Epoch 127/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6594 - acc: 0.7817 - val_loss: 0.8066 - val_acc: 0.5726
Epoch 128/200
1864/1864 [==============================

Epoch 181/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6156 - acc: 0.8063 - val_loss: 0.9504 - val_acc: 0.5645
Epoch 182/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6069 - acc: 0.8015 - val_loss: 0.8411 - val_acc: 0.5565
Epoch 183/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6244 - acc: 0.8074 - val_loss: 0.9386 - val_acc: 0.5484
Epoch 184/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6073 - acc: 0.8160 - val_loss: 0.8586 - val_acc: 0.5726
Epoch 185/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6322 - acc: 0.8036 - val_loss: 0.9797 - val_acc: 0.5645
Epoch 186/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6353 - acc: 0.7918 - val_loss: 0.9717 - val_acc: 0.5161
Epoch 187/200
1864/1864 [==============================] - 3s 1ms/step - loss: 0.6419 - acc: 0.7854 - val_loss: 0.8916 - val_acc: 0.5726
Epoch 188/200
1864/1864 [================

122/122 [==============================] - 0s 2ms/step - loss: 0.5398 - acc: 0.8525 - val_loss: 0.6367 - val_acc: 0.6774
Epoch 42/100
122/122 [==============================] - 0s 2ms/step - loss: 0.4886 - acc: 0.8689 - val_loss: 0.6354 - val_acc: 0.6774
Epoch 43/100
122/122 [==============================] - 0s 2ms/step - loss: 0.4384 - acc: 0.8934 - val_loss: 0.6397 - val_acc: 0.6774
Epoch 44/100
122/122 [==============================] - 0s 2ms/step - loss: 0.4098 - acc: 0.8852 - val_loss: 0.6378 - val_acc: 0.6855
Epoch 45/100
122/122 [==============================] - 0s 2ms/step - loss: 0.4745 - acc: 0.8770 - val_loss: 0.6326 - val_acc: 0.6935
Epoch 46/100
122/122 [==============================] - 0s 2ms/step - loss: 0.4965 - acc: 0.8689 - val_loss: 0.6310 - val_acc: 0.7016
Epoch 47/100
122/122 [==============================] - 0s 2ms/step - loss: 0.4370 - acc: 0.9016 - val_loss: 0.6297 - val_acc: 0.7016
Epoch 48/100
122/122 [==============================] - 0s 2ms/step - loss:

In [5]:
subj = 4

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1896 samples, validate on 108 samples
Epoch 1/200
1896/1896 [==============================] - 5s 2ms/step - loss: 1.0481 - acc: 0.4800 - val_loss: 0.6958 - val_acc: 0.4352
Epoch 2/200
1896/1896 [==============================] - 3s 2ms/step - loss: 1.0210 - acc: 0.5258 - val_loss: 0.7057 - val_acc: 0.4907
Epoch 3/200
1896/1896 [==============================] - 4s 2ms/step - loss: 0.9826 - acc: 0.5865 - val_loss: 0.6736 - val_acc: 0.4815
Epoch 4/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.9597 - acc: 0.6171 - val_loss: 0.6614 - val_acc: 0.5463
Epoch 5/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.9485 - acc: 0.6224 - val_loss: 0.6455 - val_acc: 0.5648
Epoch 6/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.9290 - acc: 0.6408 - val_loss: 0.6582 - val_acc: 0.5370
Epoch 7/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.9236 - acc: 0.6414 - val_loss: 0.6614 - val_acc: 0.5370
Epoch 8/

Epoch 61/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.7103 - acc: 0.7737 - val_loss: 0.6521 - val_acc: 0.6389
Epoch 62/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.7314 - acc: 0.7674 - val_loss: 0.6723 - val_acc: 0.6481
Epoch 63/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.7084 - acc: 0.7642 - val_loss: 0.6554 - val_acc: 0.6667
Epoch 64/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6985 - acc: 0.7780 - val_loss: 0.6622 - val_acc: 0.6759
Epoch 65/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.7170 - acc: 0.7616 - val_loss: 0.6513 - val_acc: 0.6852
Epoch 66/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6896 - acc: 0.7927 - val_loss: 0.6517 - val_acc: 0.6574
Epoch 67/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6828 - acc: 0.7869 - val_loss: 0.6589 - val_acc: 0.6759
Epoch 68/200
1896/1896 [========================

1896/1896 [==============================] - 3s 1ms/step - loss: 0.6492 - acc: 0.7890 - val_loss: 0.7154 - val_acc: 0.6944
Epoch 122/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6414 - acc: 0.8143 - val_loss: 0.7666 - val_acc: 0.6481
Epoch 123/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6435 - acc: 0.7811 - val_loss: 0.7668 - val_acc: 0.6389
Epoch 124/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6515 - acc: 0.7969 - val_loss: 0.7337 - val_acc: 0.6667
Epoch 125/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6275 - acc: 0.7980 - val_loss: 0.7428 - val_acc: 0.6389
Epoch 126/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6241 - acc: 0.8101 - val_loss: 0.7509 - val_acc: 0.6296
Epoch 127/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6455 - acc: 0.7985 - val_loss: 0.7619 - val_acc: 0.6574
Epoch 128/200
1896/1896 [==============================

Epoch 181/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6236 - acc: 0.7996 - val_loss: 0.7819 - val_acc: 0.6574
Epoch 182/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6055 - acc: 0.8101 - val_loss: 0.7764 - val_acc: 0.6852
Epoch 183/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6140 - acc: 0.7922 - val_loss: 0.7866 - val_acc: 0.6667
Epoch 184/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6280 - acc: 0.8022 - val_loss: 0.7967 - val_acc: 0.6481
Epoch 185/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6270 - acc: 0.8043 - val_loss: 0.7734 - val_acc: 0.6759
Epoch 186/200
1896/1896 [==============================] - 3s 1ms/step - loss: 0.6224 - acc: 0.8027 - val_loss: 0.8211 - val_acc: 0.6204
Epoch 187/200
1896/1896 [==============================] - 3s 2ms/step - loss: 0.6124 - acc: 0.8043 - val_loss: 0.7689 - val_acc: 0.6852
Epoch 188/200
1896/1896 [================

106/106 [==============================] - 0s 2ms/step - loss: 0.6145 - acc: 0.8113 - val_loss: 0.7501 - val_acc: 0.6204
Epoch 42/100
106/106 [==============================] - 0s 2ms/step - loss: 0.6595 - acc: 0.7925 - val_loss: 0.7518 - val_acc: 0.6204
Epoch 43/100
106/106 [==============================] - 0s 2ms/step - loss: 0.5681 - acc: 0.8396 - val_loss: 0.7529 - val_acc: 0.6204
Epoch 44/100
106/106 [==============================] - 0s 2ms/step - loss: 0.5315 - acc: 0.8113 - val_loss: 0.7572 - val_acc: 0.6111
Epoch 45/100
106/106 [==============================] - 0s 2ms/step - loss: 0.5623 - acc: 0.8302 - val_loss: 0.7630 - val_acc: 0.6111
Epoch 46/100
106/106 [==============================] - 0s 2ms/step - loss: 0.5227 - acc: 0.8208 - val_loss: 0.7662 - val_acc: 0.6019
Epoch 47/100
106/106 [==============================] - 0s 2ms/step - loss: 0.5240 - acc: 0.8302 - val_loss: 0.7707 - val_acc: 0.6019
Epoch 48/100
106/106 [==============================] - 0s 2ms/step - loss:

In [6]:
subj = 5

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1889 samples, validate on 111 samples
Epoch 1/200
1889/1889 [==============================] - 4s 2ms/step - loss: 1.0636 - acc: 0.4584 - val_loss: 0.7005 - val_acc: 0.4595
Epoch 2/200
1889/1889 [==============================] - 3s 2ms/step - loss: 1.0340 - acc: 0.4584 - val_loss: 0.7150 - val_acc: 0.4685
Epoch 3/200
1889/1889 [==============================] - 3s 2ms/step - loss: 1.0120 - acc: 0.5368 - val_loss: 0.6925 - val_acc: 0.6216
Epoch 4/200
1889/1889 [==============================] - 3s 2ms/step - loss: 0.9773 - acc: 0.5807 - val_loss: 0.6376 - val_acc: 0.6847
Epoch 5/200
1889/1889 [==============================] - 3s 2ms/step - loss: 0.9621 - acc: 0.6289 - val_loss: 0.6716 - val_acc: 0.6306
Epoch 6/200
1889/1889 [==============================] - 3s 2ms/step - loss: 0.9496 - acc: 0.6093 - val_loss: 0.6523 - val_acc: 0.6486
Epoch 7/200
1889/1889 [==============================] - 3s 2ms/step - loss: 0.9235 - acc: 0.6527 - val_loss: 0.6577 - val_acc: 0.6486
Epoch 8/

Epoch 61/200
1889/1889 [==============================] - 3s 2ms/step - loss: 0.7247 - acc: 0.7581 - val_loss: 0.7820 - val_acc: 0.5946
Epoch 62/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.7210 - acc: 0.7623 - val_loss: 0.7592 - val_acc: 0.5856
Epoch 63/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.7443 - acc: 0.7480 - val_loss: 0.7767 - val_acc: 0.6036
Epoch 64/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.7316 - acc: 0.7692 - val_loss: 0.7908 - val_acc: 0.5676
Epoch 65/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.7544 - acc: 0.7570 - val_loss: 0.8056 - val_acc: 0.5856
Epoch 66/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.7271 - acc: 0.7734 - val_loss: 0.7224 - val_acc: 0.6126
Epoch 67/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.7207 - acc: 0.7692 - val_loss: 0.7586 - val_acc: 0.6126
Epoch 68/200
1889/1889 [========================

1889/1889 [==============================] - 3s 1ms/step - loss: 0.6867 - acc: 0.7872 - val_loss: 0.9259 - val_acc: 0.5766
Epoch 122/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6702 - acc: 0.7787 - val_loss: 0.8640 - val_acc: 0.5946
Epoch 123/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6805 - acc: 0.7771 - val_loss: 0.8197 - val_acc: 0.6126
Epoch 124/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6707 - acc: 0.7771 - val_loss: 0.9680 - val_acc: 0.5495
Epoch 125/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6699 - acc: 0.7777 - val_loss: 0.8501 - val_acc: 0.5766
Epoch 126/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6941 - acc: 0.7618 - val_loss: 0.8839 - val_acc: 0.5946
Epoch 127/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6511 - acc: 0.8010 - val_loss: 0.8754 - val_acc: 0.5856
Epoch 128/200
1889/1889 [==============================

Epoch 181/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6668 - acc: 0.7983 - val_loss: 0.8740 - val_acc: 0.5766
Epoch 182/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6303 - acc: 0.7962 - val_loss: 0.8827 - val_acc: 0.5586
Epoch 183/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6645 - acc: 0.7840 - val_loss: 0.9478 - val_acc: 0.5495
Epoch 184/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6582 - acc: 0.7888 - val_loss: 0.8059 - val_acc: 0.5676
Epoch 185/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6601 - acc: 0.7920 - val_loss: 0.8707 - val_acc: 0.5676
Epoch 186/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6497 - acc: 0.7830 - val_loss: 0.9225 - val_acc: 0.5676
Epoch 187/200
1889/1889 [==============================] - 3s 1ms/step - loss: 0.6325 - acc: 0.8010 - val_loss: 0.8614 - val_acc: 0.5495
Epoch 188/200
1889/1889 [================

110/110 [==============================] - 0s 2ms/step - loss: 0.4144 - acc: 0.9091 - val_loss: 0.6336 - val_acc: 0.7387
Epoch 42/100
110/110 [==============================] - 0s 2ms/step - loss: 0.4122 - acc: 0.9273 - val_loss: 0.6317 - val_acc: 0.7477
Epoch 43/100
110/110 [==============================] - 0s 2ms/step - loss: 0.4645 - acc: 0.9000 - val_loss: 0.6300 - val_acc: 0.7477
Epoch 44/100
110/110 [==============================] - 0s 2ms/step - loss: 0.4427 - acc: 0.9273 - val_loss: 0.6279 - val_acc: 0.7477
Epoch 45/100
110/110 [==============================] - 0s 2ms/step - loss: 0.4267 - acc: 0.8727 - val_loss: 0.6254 - val_acc: 0.7477
Epoch 46/100
110/110 [==============================] - 0s 2ms/step - loss: 0.5034 - acc: 0.8909 - val_loss: 0.6266 - val_acc: 0.7477
Epoch 47/100
110/110 [==============================] - 0s 2ms/step - loss: 0.4412 - acc: 0.8636 - val_loss: 0.6309 - val_acc: 0.7387
Epoch 48/100
110/110 [==============================] - 0s 2ms/step - loss:

In [12]:
subj = 6

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1890 samples, validate on 110 samples
Epoch 1/200
1890/1890 [==============================] - 6s 3ms/step - loss: 1.0531 - acc: 0.5037 - val_loss: 0.7053 - val_acc: 0.4273
Epoch 2/200
1890/1890 [==============================] - 3s 2ms/step - loss: 1.0169 - acc: 0.5095 - val_loss: 0.7243 - val_acc: 0.4364
Epoch 3/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.9813 - acc: 0.5878 - val_loss: 0.7353 - val_acc: 0.4909
Epoch 4/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.9688 - acc: 0.6270 - val_loss: 0.7416 - val_acc: 0.5182
Epoch 5/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.9460 - acc: 0.6238 - val_loss: 0.7744 - val_acc: 0.4727
Epoch 6/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.9330 - acc: 0.6513 - val_loss: 0.7716 - val_acc: 0.4636
Epoch 7/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.9356 - acc: 0.6370 - val_loss: 0.7653 - val_acc: 0.4818
Epoch 8/

Epoch 61/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.7440 - acc: 0.7534 - val_loss: 0.7988 - val_acc: 0.4818
Epoch 62/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.7157 - acc: 0.7772 - val_loss: 0.8197 - val_acc: 0.4909
Epoch 63/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.7428 - acc: 0.7444 - val_loss: 0.7826 - val_acc: 0.5182
Epoch 64/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.7312 - acc: 0.7571 - val_loss: 0.7862 - val_acc: 0.5182
Epoch 65/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.7140 - acc: 0.7550 - val_loss: 0.8061 - val_acc: 0.5000
Epoch 66/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.7426 - acc: 0.7513 - val_loss: 0.7801 - val_acc: 0.5182
Epoch 67/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.7398 - acc: 0.7519 - val_loss: 0.7723 - val_acc: 0.5273
Epoch 68/200
1890/1890 [========================

1890/1890 [==============================] - 3s 1ms/step - loss: 0.6873 - acc: 0.7772 - val_loss: 0.7621 - val_acc: 0.5909
Epoch 122/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.6674 - acc: 0.7767 - val_loss: 0.7823 - val_acc: 0.5727
Epoch 123/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.6973 - acc: 0.7783 - val_loss: 0.7935 - val_acc: 0.5364
Epoch 124/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.7070 - acc: 0.7614 - val_loss: 0.8149 - val_acc: 0.5182
Epoch 125/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.6860 - acc: 0.7683 - val_loss: 0.7592 - val_acc: 0.5727
Epoch 126/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.6987 - acc: 0.7667 - val_loss: 0.7847 - val_acc: 0.5545
Epoch 127/200
1890/1890 [==============================] - 3s 1ms/step - loss: 0.6834 - acc: 0.7831 - val_loss: 0.8254 - val_acc: 0.5273
Epoch 128/200
1890/1890 [==============================

Epoch 181/200
1890/1890 [==============================] - 3s 2ms/step - loss: 0.6561 - acc: 0.7942 - val_loss: 0.7881 - val_acc: 0.5909
Epoch 182/200
1890/1890 [==============================] - 3s 2ms/step - loss: 0.6567 - acc: 0.7873 - val_loss: 0.8214 - val_acc: 0.5455
Epoch 183/200
1890/1890 [==============================] - 3s 2ms/step - loss: 0.6632 - acc: 0.7899 - val_loss: 0.7898 - val_acc: 0.5636
Epoch 184/200
1890/1890 [==============================] - 3s 2ms/step - loss: 0.6929 - acc: 0.7778 - val_loss: 0.7892 - val_acc: 0.5727
Epoch 185/200
1890/1890 [==============================] - 3s 2ms/step - loss: 0.6515 - acc: 0.7894 - val_loss: 0.8095 - val_acc: 0.5364
Epoch 186/200
1890/1890 [==============================] - 3s 2ms/step - loss: 0.6788 - acc: 0.7783 - val_loss: 0.7654 - val_acc: 0.5818
Epoch 187/200
1890/1890 [==============================] - 3s 2ms/step - loss: 0.6610 - acc: 0.7968 - val_loss: 0.8202 - val_acc: 0.5364
Epoch 188/200
1890/1890 [================

110/110 [==============================] - 0s 2ms/step - loss: 0.5457 - acc: 0.8545 - val_loss: 0.7989 - val_acc: 0.5545
Epoch 42/100
110/110 [==============================] - 0s 2ms/step - loss: 0.5569 - acc: 0.8636 - val_loss: 0.7998 - val_acc: 0.5545
Epoch 43/100
110/110 [==============================] - 0s 2ms/step - loss: 0.4710 - acc: 0.8818 - val_loss: 0.7989 - val_acc: 0.5545
Epoch 44/100
110/110 [==============================] - 0s 2ms/step - loss: 0.4831 - acc: 0.8727 - val_loss: 0.7957 - val_acc: 0.5636
Epoch 45/100
110/110 [==============================] - 0s 2ms/step - loss: 0.5804 - acc: 0.8000 - val_loss: 0.7927 - val_acc: 0.5636
Epoch 46/100
110/110 [==============================] - 0s 2ms/step - loss: 0.5085 - acc: 0.8455 - val_loss: 0.7899 - val_acc: 0.5545
Epoch 47/100
110/110 [==============================] - 0s 2ms/step - loss: 0.5057 - acc: 0.8818 - val_loss: 0.7919 - val_acc: 0.5545
Epoch 48/100
110/110 [==============================] - 0s 2ms/step - loss:

In [8]:
subj = 7

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1966 samples, validate on 72 samples
Epoch 1/200
1966/1966 [==============================] - 5s 2ms/step - loss: 1.0408 - acc: 0.4934 - val_loss: 0.7116 - val_acc: 0.3889
Epoch 2/200
1966/1966 [==============================] - 3s 1ms/step - loss: 1.0090 - acc: 0.5392 - val_loss: 0.7129 - val_acc: 0.5417
Epoch 3/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.9831 - acc: 0.6165 - val_loss: 0.8050 - val_acc: 0.3611
Epoch 4/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.9629 - acc: 0.6114 - val_loss: 0.7706 - val_acc: 0.4306
Epoch 5/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.9497 - acc: 0.6353 - val_loss: 0.7989 - val_acc: 0.4306
Epoch 6/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.9359 - acc: 0.6526 - val_loss: 0.8162 - val_acc: 0.4167
Epoch 7/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.9244 - acc: 0.6521 - val_loss: 0.7971 - val_acc: 0.3750
Epoch 8/2

Epoch 61/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7532 - acc: 0.7558 - val_loss: 0.8310 - val_acc: 0.4583
Epoch 62/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7410 - acc: 0.7594 - val_loss: 0.8149 - val_acc: 0.4722
Epoch 63/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7572 - acc: 0.7630 - val_loss: 0.7971 - val_acc: 0.5278
Epoch 64/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7361 - acc: 0.7645 - val_loss: 0.8150 - val_acc: 0.5139
Epoch 65/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7337 - acc: 0.7670 - val_loss: 0.7908 - val_acc: 0.5278
Epoch 66/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7343 - acc: 0.7609 - val_loss: 0.7900 - val_acc: 0.5000
Epoch 67/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7498 - acc: 0.7660 - val_loss: 0.8371 - val_acc: 0.5000
Epoch 68/200
1966/1966 [========================

1966/1966 [==============================] - 3s 1ms/step - loss: 0.6830 - acc: 0.7864 - val_loss: 0.8812 - val_acc: 0.5000
Epoch 122/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6791 - acc: 0.7838 - val_loss: 0.9607 - val_acc: 0.5000
Epoch 123/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6986 - acc: 0.7721 - val_loss: 0.9609 - val_acc: 0.5000
Epoch 124/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6852 - acc: 0.7874 - val_loss: 0.8741 - val_acc: 0.5278
Epoch 125/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7003 - acc: 0.7828 - val_loss: 0.8884 - val_acc: 0.5139
Epoch 126/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6931 - acc: 0.7772 - val_loss: 0.8538 - val_acc: 0.5139
Epoch 127/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.7103 - acc: 0.7726 - val_loss: 0.8658 - val_acc: 0.5139
Epoch 128/200
1966/1966 [==============================

Epoch 181/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6532 - acc: 0.8042 - val_loss: 0.9460 - val_acc: 0.5000
Epoch 182/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6599 - acc: 0.7955 - val_loss: 0.8755 - val_acc: 0.4861
Epoch 183/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6773 - acc: 0.7792 - val_loss: 0.9709 - val_acc: 0.5139
Epoch 184/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6807 - acc: 0.7874 - val_loss: 0.9288 - val_acc: 0.4722
Epoch 185/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6771 - acc: 0.7894 - val_loss: 0.9057 - val_acc: 0.5278
Epoch 186/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6609 - acc: 0.7945 - val_loss: 0.9329 - val_acc: 0.4861
Epoch 187/200
1966/1966 [==============================] - 3s 1ms/step - loss: 0.6640 - acc: 0.7945 - val_loss: 0.8748 - val_acc: 0.5417
Epoch 188/200
1966/1966 [================

Epoch 42/100
72/72 [==============================] - 0s 2ms/step - loss: 0.4283 - acc: 0.8889 - val_loss: 0.6629 - val_acc: 0.6806
Epoch 43/100
72/72 [==============================] - 0s 2ms/step - loss: 0.3793 - acc: 0.9028 - val_loss: 0.6630 - val_acc: 0.6667
Epoch 44/100
72/72 [==============================] - 0s 2ms/step - loss: 0.3703 - acc: 0.9167 - val_loss: 0.6625 - val_acc: 0.6667
Epoch 45/100
72/72 [==============================] - 0s 2ms/step - loss: 0.4526 - acc: 0.9028 - val_loss: 0.6621 - val_acc: 0.6667
Epoch 46/100
72/72 [==============================] - 0s 2ms/step - loss: 0.3945 - acc: 0.8889 - val_loss: 0.6610 - val_acc: 0.6667
Epoch 47/100
72/72 [==============================] - 0s 2ms/step - loss: 0.4132 - acc: 0.9028 - val_loss: 0.6598 - val_acc: 0.6667
Epoch 48/100
72/72 [==============================] - 0s 2ms/step - loss: 0.3466 - acc: 0.9167 - val_loss: 0.6590 - val_acc: 0.6667
Epoch 49/100
72/72 [==============================] - 0s 2ms/step - loss: 0.

In [9]:
subj = 8

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1853 samples, validate on 129 samples
Epoch 1/200
1853/1853 [==============================] - 5s 3ms/step - loss: 1.0565 - acc: 0.4641 - val_loss: 0.6943 - val_acc: 0.5039
Epoch 2/200
1853/1853 [==============================] - 3s 1ms/step - loss: 1.0381 - acc: 0.4873 - val_loss: 0.6711 - val_acc: 0.6744
Epoch 3/200
1853/1853 [==============================] - 3s 1ms/step - loss: 1.0084 - acc: 0.5467 - val_loss: 0.6453 - val_acc: 0.6434
Epoch 4/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.9871 - acc: 0.6087 - val_loss: 0.5688 - val_acc: 0.7829
Epoch 5/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.9671 - acc: 0.6071 - val_loss: 0.5831 - val_acc: 0.7209
Epoch 6/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.9565 - acc: 0.6395 - val_loss: 0.5522 - val_acc: 0.7519
Epoch 7/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.9435 - acc: 0.6546 - val_loss: 0.6013 - val_acc: 0.6899
Epoch 8/

Epoch 61/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7688 - acc: 0.7361 - val_loss: 0.6347 - val_acc: 0.6512
Epoch 62/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7650 - acc: 0.7474 - val_loss: 0.6581 - val_acc: 0.6434
Epoch 63/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7565 - acc: 0.7480 - val_loss: 0.5837 - val_acc: 0.6589
Epoch 64/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7714 - acc: 0.7507 - val_loss: 0.6437 - val_acc: 0.6589
Epoch 65/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7649 - acc: 0.7399 - val_loss: 0.6353 - val_acc: 0.6667
Epoch 66/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7526 - acc: 0.7539 - val_loss: 0.6878 - val_acc: 0.6279
Epoch 67/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7440 - acc: 0.7652 - val_loss: 0.6575 - val_acc: 0.6279
Epoch 68/200
1853/1853 [========================

1853/1853 [==============================] - 3s 1ms/step - loss: 0.7000 - acc: 0.7744 - val_loss: 0.6613 - val_acc: 0.6512
Epoch 122/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7177 - acc: 0.7642 - val_loss: 0.6867 - val_acc: 0.6357
Epoch 123/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7049 - acc: 0.7755 - val_loss: 0.7243 - val_acc: 0.5969
Epoch 124/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7171 - acc: 0.7620 - val_loss: 0.6894 - val_acc: 0.6512
Epoch 125/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6958 - acc: 0.7804 - val_loss: 0.6795 - val_acc: 0.6589
Epoch 126/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.7186 - acc: 0.7685 - val_loss: 0.6330 - val_acc: 0.6667
Epoch 127/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6984 - acc: 0.7696 - val_loss: 0.6759 - val_acc: 0.6512
Epoch 128/200
1853/1853 [==============================

Epoch 181/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6835 - acc: 0.7771 - val_loss: 0.6749 - val_acc: 0.6512
Epoch 182/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6782 - acc: 0.7863 - val_loss: 0.7054 - val_acc: 0.6589
Epoch 183/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6976 - acc: 0.7723 - val_loss: 0.7025 - val_acc: 0.6434
Epoch 184/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6538 - acc: 0.7922 - val_loss: 0.6646 - val_acc: 0.6744
Epoch 185/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6846 - acc: 0.7744 - val_loss: 0.6229 - val_acc: 0.6822
Epoch 186/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6782 - acc: 0.7739 - val_loss: 0.6382 - val_acc: 0.6589
Epoch 187/200
1853/1853 [==============================] - 3s 1ms/step - loss: 0.6986 - acc: 0.7728 - val_loss: 0.6328 - val_acc: 0.6977
Epoch 188/200
1853/1853 [================

128/128 [==============================] - 0s 2ms/step - loss: 0.2763 - acc: 0.9453 - val_loss: 0.5255 - val_acc: 0.7519
Epoch 42/100
128/128 [==============================] - 0s 2ms/step - loss: 0.2892 - acc: 0.9375 - val_loss: 0.5280 - val_acc: 0.7364
Epoch 43/100
128/128 [==============================] - 0s 2ms/step - loss: 0.2787 - acc: 0.9375 - val_loss: 0.5279 - val_acc: 0.7364
Epoch 44/100
128/128 [==============================] - 0s 2ms/step - loss: 0.3040 - acc: 0.9375 - val_loss: 0.5235 - val_acc: 0.7364
Epoch 45/100
128/128 [==============================] - 0s 2ms/step - loss: 0.3524 - acc: 0.9531 - val_loss: 0.5262 - val_acc: 0.7364
Epoch 46/100
128/128 [==============================] - 0s 2ms/step - loss: 0.2745 - acc: 0.9453 - val_loss: 0.5324 - val_acc: 0.7287
Epoch 47/100
128/128 [==============================] - 0s 2ms/step - loss: 0.2466 - acc: 0.9141 - val_loss: 0.5336 - val_acc: 0.7287
Epoch 48/100
128/128 [==============================] - 0s 2ms/step - loss:

In [10]:
subj = 9

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == subj:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 0:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1860 samples, validate on 126 samples
Epoch 1/200
1860/1860 [==============================] - 5s 3ms/step - loss: 1.0525 - acc: 0.4919 - val_loss: 0.6951 - val_acc: 0.4921
Epoch 2/200
1860/1860 [==============================] - 3s 1ms/step - loss: 1.0000 - acc: 0.5382 - val_loss: 0.6906 - val_acc: 0.5794
Epoch 3/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.9547 - acc: 0.6269 - val_loss: 0.6851 - val_acc: 0.6111
Epoch 4/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.9364 - acc: 0.6344 - val_loss: 0.6925 - val_acc: 0.6111
Epoch 5/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.9035 - acc: 0.6672 - val_loss: 0.6648 - val_acc: 0.6587
Epoch 6/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.9004 - acc: 0.6763 - val_loss: 0.6929 - val_acc: 0.6270
Epoch 7/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.8964 - acc: 0.6720 - val_loss: 0.6918 - val_acc: 0.6270
Epoch 8/

Epoch 61/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.7015 - acc: 0.7758 - val_loss: 0.7398 - val_acc: 0.6667
Epoch 62/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.7369 - acc: 0.7672 - val_loss: 0.7795 - val_acc: 0.6349
Epoch 63/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.7244 - acc: 0.7715 - val_loss: 0.7918 - val_acc: 0.6508
Epoch 64/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.7141 - acc: 0.7774 - val_loss: 0.7653 - val_acc: 0.6587
Epoch 65/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.7094 - acc: 0.7817 - val_loss: 0.7714 - val_acc: 0.6587
Epoch 66/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.7052 - acc: 0.7769 - val_loss: 0.7644 - val_acc: 0.6587
Epoch 67/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6923 - acc: 0.7806 - val_loss: 0.7871 - val_acc: 0.6429
Epoch 68/200
1860/1860 [========================

Epoch 121/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6804 - acc: 0.7860 - val_loss: 0.8119 - val_acc: 0.6032
Epoch 122/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6476 - acc: 0.7973 - val_loss: 0.7915 - val_acc: 0.6429
Epoch 123/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6582 - acc: 0.7935 - val_loss: 0.7824 - val_acc: 0.6270
Epoch 124/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6589 - acc: 0.7952 - val_loss: 0.8246 - val_acc: 0.5873
Epoch 125/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6607 - acc: 0.7855 - val_loss: 0.8225 - val_acc: 0.5952
Epoch 126/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6658 - acc: 0.7903 - val_loss: 0.8183 - val_acc: 0.5873
Epoch 127/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6628 - acc: 0.7823 - val_loss: 0.8112 - val_acc: 0.6349
Epoch 128/200
1860/1860 [================

Epoch 181/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6162 - acc: 0.8086 - val_loss: 0.8771 - val_acc: 0.5714
Epoch 182/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6153 - acc: 0.8167 - val_loss: 0.8771 - val_acc: 0.6190
Epoch 183/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.5963 - acc: 0.8226 - val_loss: 0.8757 - val_acc: 0.6111
Epoch 184/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6162 - acc: 0.8081 - val_loss: 0.9057 - val_acc: 0.5952
Epoch 185/200
1860/1860 [==============================] - 3s 2ms/step - loss: 0.6081 - acc: 0.8086 - val_loss: 0.8894 - val_acc: 0.6667
Epoch 186/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6217 - acc: 0.8091 - val_loss: 0.8562 - val_acc: 0.6587
Epoch 187/200
1860/1860 [==============================] - 3s 1ms/step - loss: 0.6066 - acc: 0.8172 - val_loss: 0.8760 - val_acc: 0.6349
Epoch 188/200
1860/1860 [================

124/124 [==============================] - 0s 2ms/step - loss: 0.5916 - acc: 0.7823 - val_loss: 0.6732 - val_acc: 0.6667
Epoch 42/100
124/124 [==============================] - 0s 2ms/step - loss: 0.5681 - acc: 0.8468 - val_loss: 0.6720 - val_acc: 0.6746
Epoch 43/100
124/124 [==============================] - 0s 2ms/step - loss: 0.5726 - acc: 0.8145 - val_loss: 0.6721 - val_acc: 0.6746
Epoch 44/100
124/124 [==============================] - 0s 2ms/step - loss: 0.5763 - acc: 0.8226 - val_loss: 0.6845 - val_acc: 0.6667
Epoch 45/100
124/124 [==============================] - 0s 2ms/step - loss: 0.5856 - acc: 0.8065 - val_loss: 0.6815 - val_acc: 0.6667
Epoch 46/100
124/124 [==============================] - 0s 2ms/step - loss: 0.6161 - acc: 0.7903 - val_loss: 0.6799 - val_acc: 0.6667
Epoch 47/100
124/124 [==============================] - 0s 2ms/step - loss: 0.5199 - acc: 0.8387 - val_loss: 0.6773 - val_acc: 0.6667
Epoch 48/100
124/124 [==============================] - 0s 2ms/step - loss:

In [11]:
subj = 9

for i in range(0,len(file_list)):
    data = loadmat(file_list[i])
    eeg_matrix = data['eeg_matrix_norm']
    # Try cutting the epoch down from -0.5-1.5s to 0-1.5s
    eeg_matrix = eeg_matrix[:,:,64:256]
    label_array = data['label_array']
    labels = label_array.flatten()
    label = np.zeros((len(label_array),2))
    target_ind = labels == 1
    label[target_ind,1] = 1
    label[~target_ind,0] = 1
    if i == 0:
        eeg_matrix_test = eeg_matrix[0:int(round(eeg_matrix.shape[0]/2)),:,:]
        label_test = label[0:int(round(label.shape[0]/2))]
        eeg_matrix_validate = eeg_matrix[int(round(eeg_matrix.shape[0]/2)):-1,:,:]
        label_validate = label[int(round(eeg_matrix.shape[0]/2)):-1]
    elif i == 1:
        eeg_matrix_train = eeg_matrix
        label_train = label
    else:
        eeg_matrix_train = np.concatenate((eeg_matrix_train,eeg_matrix),axis=0)
        label_train = np.concatenate((label_train,label),axis=0)
eeg_matrix_train = np.expand_dims(eeg_matrix_train,axis=1)
eeg_matrix_test = np.expand_dims(eeg_matrix_test,axis=1)
eeg_matrix_validate = np.expand_dims(eeg_matrix_validate,axis=1)
model = EEGNet(nb_classes = 2, Chans = 64, Samples = 192, dropoutRate = 0.35, gaussianNoise=False)
var_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
class_weights = {0:1, 1:3}
model.fit(x = eeg_matrix_train, y = label_train, batch_size = 16, epochs = 200, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)
var_adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=var_adam, metrics=['accuracy'])
model.fit(x = eeg_matrix_validate, y = label_validate, batch_size = 16, epochs = 100, validation_data =(eeg_matrix_test,label_test), class_weight = class_weights)

Train on 1930 samples, validate on 90 samples
Epoch 1/200
1930/1930 [==============================] - 5s 3ms/step - loss: 1.0573 - acc: 0.4813 - val_loss: 0.7010 - val_acc: 0.3889
Epoch 2/200
1930/1930 [==============================] - 3s 1ms/step - loss: 1.0366 - acc: 0.4995 - val_loss: 0.6981 - val_acc: 0.4667
Epoch 3/200
1930/1930 [==============================] - 3s 1ms/step - loss: 1.0023 - acc: 0.5466 - val_loss: 0.7034 - val_acc: 0.5111
Epoch 4/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.9740 - acc: 0.6166 - val_loss: 0.7186 - val_acc: 0.5667
Epoch 5/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.9574 - acc: 0.6150 - val_loss: 0.7216 - val_acc: 0.5667
Epoch 6/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.9381 - acc: 0.6389 - val_loss: 0.7022 - val_acc: 0.5889
Epoch 7/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.9303 - acc: 0.6404 - val_loss: 0.7564 - val_acc: 0.5333
Epoch 8/2

Epoch 61/200
1930/1930 [==============================] - 3s 2ms/step - loss: 0.7475 - acc: 0.7642 - val_loss: 0.8912 - val_acc: 0.6111
Epoch 62/200
1930/1930 [==============================] - 3s 2ms/step - loss: 0.7562 - acc: 0.7451 - val_loss: 0.8417 - val_acc: 0.6444
Epoch 63/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.7694 - acc: 0.7472 - val_loss: 0.8505 - val_acc: 0.6111
Epoch 64/200
1930/1930 [==============================] - 4s 2ms/step - loss: 0.7571 - acc: 0.7554 - val_loss: 0.8735 - val_acc: 0.6000
Epoch 65/200
1930/1930 [==============================] - 3s 2ms/step - loss: 0.7423 - acc: 0.7560 - val_loss: 0.8478 - val_acc: 0.6111
Epoch 66/200
1930/1930 [==============================] - 4s 2ms/step - loss: 0.7524 - acc: 0.7534 - val_loss: 0.8444 - val_acc: 0.6444
Epoch 67/200
1930/1930 [==============================] - 3s 2ms/step - loss: 0.7521 - acc: 0.7513 - val_loss: 0.8322 - val_acc: 0.6444
Epoch 68/200
1930/1930 [========================

1930/1930 [==============================] - 3s 1ms/step - loss: 0.6853 - acc: 0.7648 - val_loss: 0.9303 - val_acc: 0.6000
Epoch 122/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6951 - acc: 0.7741 - val_loss: 0.9152 - val_acc: 0.6333
Epoch 123/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6769 - acc: 0.7793 - val_loss: 0.8808 - val_acc: 0.6000
Epoch 124/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.7043 - acc: 0.7725 - val_loss: 0.8748 - val_acc: 0.6222
Epoch 125/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.7040 - acc: 0.7627 - val_loss: 0.9790 - val_acc: 0.6222
Epoch 126/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6886 - acc: 0.7756 - val_loss: 0.9212 - val_acc: 0.6000
Epoch 127/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6656 - acc: 0.7881 - val_loss: 0.9253 - val_acc: 0.6222
Epoch 128/200
1930/1930 [==============================

Epoch 181/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6872 - acc: 0.7798 - val_loss: 0.9843 - val_acc: 0.5667
Epoch 182/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6668 - acc: 0.7933 - val_loss: 0.9624 - val_acc: 0.5444
Epoch 183/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6913 - acc: 0.7912 - val_loss: 0.9709 - val_acc: 0.6000
Epoch 184/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6633 - acc: 0.7860 - val_loss: 1.0087 - val_acc: 0.5556
Epoch 185/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6658 - acc: 0.7845 - val_loss: 1.0340 - val_acc: 0.5222
Epoch 186/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6539 - acc: 0.7865 - val_loss: 1.0054 - val_acc: 0.5667
Epoch 187/200
1930/1930 [==============================] - 3s 1ms/step - loss: 0.6462 - acc: 0.7819 - val_loss: 1.0108 - val_acc: 0.5889
Epoch 188/200
1930/1930 [================

90/90 [==============================] - 0s 2ms/step - loss: 0.5766 - acc: 0.8444 - val_loss: 0.6699 - val_acc: 0.6778
Epoch 43/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6300 - acc: 0.8000 - val_loss: 0.6664 - val_acc: 0.6778
Epoch 44/100
90/90 [==============================] - 0s 2ms/step - loss: 0.5309 - acc: 0.8333 - val_loss: 0.6627 - val_acc: 0.6778
Epoch 45/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6177 - acc: 0.8444 - val_loss: 0.6591 - val_acc: 0.6889
Epoch 46/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6129 - acc: 0.8111 - val_loss: 0.6537 - val_acc: 0.7111
Epoch 47/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6265 - acc: 0.7556 - val_loss: 0.6468 - val_acc: 0.7111
Epoch 48/100
90/90 [==============================] - 0s 2ms/step - loss: 0.5407 - acc: 0.8556 - val_loss: 0.6399 - val_acc: 0.6889
Epoch 49/100
90/90 [==============================] - 0s 2ms/step - loss: 0.5491 - acc: 0